<a href="https://colab.research.google.com/github/KimGriezmann/DeepLearning_BeTheLegend/blob/master/pre_processing_function/%EC%B5%9C%EA%B7%BC%205%EC%9D%BC(%EB%B3%80%EA%B2%BD%20%EA%B0%80%EB%8A%A5)%20%EA%B2%BD%EA%B8%B0%EC%9D%98%20%ED%83%80%EC%9C%A8%20%EA%B5%AC%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import pandas as pd
import tqdm

In [29]:
data = pd.read_csv("/content/data_2018.csv", encoding='euc-kr')
hitter = pd.read_csv("/content/batter_data_2018.csv", encoding='euc-kr')

In [30]:
data = pd.DataFrame(data)
hitter = pd.DataFrame(hitter)

In [31]:
print(data.head(), "\n길이 : ", len(data) ,"\n")
print(hitter.head(), "\n길이 :", len(hitter))

  name         day  vs result  bat_order  ...    ops  pitch  avLI  RE24    WPA
0  나성범  2018-03-24  LG  W 4:2          3  ...  0.000     12  1.49 -1.27 -0.100
1  나성범  2018-03-25  LG  W 7:1          3  ...  0.476     15  0.90  1.42  0.113
2  나성범  2018-03-27  한화  W 9:6          3  ...  0.930     15  0.93  4.22  0.329
3  나성범  2018-03-28  한화  L 2:6          3  ...  0.694      9  0.87 -1.09 -0.087
4  나성범  2018-03-29  한화  W 4:1          3  ...  0.754     15  1.60 -0.19 -0.076

[5 rows x 32 columns] 
길이 :  11316 

  Team  name
0   NC   나성범
1   NC  스크럭스
2   NC   박민우
3   NC   노진혁
4   NC   권희동 
길이 : 90


In [32]:
# 날짜 데이터 값, 형식 변경
# 현재 : object 형식으로 "2018-04-04" 값으로 존재 (의미 : 4월 4일)
# 변경 : int 형식으로 20180404 으로 변경 (의미 : 2018년 4월 4일)

def day_to_int(data):
  data['day'] = pd.to_numeric(data['day'].str.replace("-",""))
  return data

In [33]:
# 날짜 데이터 문장열 형식의 값에서 -> 정수형식의 값으로 변경
data = day_to_int(data)

In [34]:
data.loc[:,['day','name','ab','hit']].dtypes

day      int64
name    object
ab       int64
hit      int64
dtype: object

0        False
1        False
2        False
3        False
4        False
         ...  
11311    False
11312    False
11313    False
11314    False
11315    False
Length: 11316, dtype: bool

In [35]:
# 더블헤더 처리
temp = np.zeros(len(data))

# day, name이 같은 값이 있다는 것은 더블헤더 경기가 있다는것
# keep=False 옵션은 중복 값에 해당하는 튜플들을 모두 True로 리턴, 그 외 옵션 : 'first', 'last' 의 경우 둘 중 하나만 True
dup = data.loc[:,['day','name']].duplicated(keep=False) 

for i in range(len(data)):
  if dup[i]==False:
    temp[i]=0
  elif dup[i]==True:
    temp[i]=1

# 중복의 첫번째 행은 더블헤더 1차전
# 중복인 두번째 행이 더블헤더 2차전이다.
for i in range(1,len(temp)):
  if temp[i]==1 and temp[i-1]==1:
    temp[i]=2

data['double']=temp

In [21]:
#
# 선수마다 최근 5일의 타율을 구해야함. 
# 현재 day에서 -5부터 -1까지의 sum(hit)/sum(ab) 값을 구하면 된다

# hitter_data : 90명 타자 data
# record_data : 날짜별 타자 기록
# recent_day : 기본(최근 5일), 설정 가능
def make_recent_days(hitter_data, record_data, day=5):

  # 최근 5일(기본) 경기 타율 저장할 공간 생성
  recent_days_avg = np.zeros(len(record_data)) # 최근 5일(기본) 경기 타율
  recent_days_hitter = np.empty(len(record_data), dtype=np.chararray) # type : 문자열 저장을 위해 chararray
  recent_days_day = np.empty(len(record_data), dtype=np.int32) # 기준이 되는 날짜
  recent_days_double = np.zeros(len(record_data), dtype=np.int8) # 더블헤더 분류 / 0 : 더블헤더 아님 / 1 : 더블헤더 1차전 / 2 : 더블헤더 2차전

  ps = 0
  for h in tqdm.tqdm(hitter_data['name']): # 90명 타자 한명씩 접근
    hdata = record_data.loc[record_data['name']==h].reset_index() # 특정 타자 한명에 대해서만 데이터 추출
    for i in range(len(hdata['day'])): # 추출한 데이터에서 한 행씩 접근
      recent_days_hitter[ps] = h # 현재 타자
      r = hdata.loc[i,'day']  # 현재 기준 날짜
      recent_days_day[ps] = r
      d = hdata.loc[i,'double'] # 현재 더블헤더 분류
      recent_days_double[ps] = d      
      if d==2: # 더블헤더 2차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r].loc[hdata['double']!=2] # r 날짜까지 가져와서 2차전을 제외
      else: # 더블헤더가 아니고, 더블헤더 1차전이라면
        hrdata = hdata.loc[hdata['day']>=r-day].loc[hdata['day']<=r-1] # 접근한 행의 날짜를 기준으로 5일 경기
         
      if len(hrdata)==0: # 최근 5일 경기 없으면 넘어가고
        ps+=1
        continue
      else : # 있으면
        sum_ab = sum(hrdata['ab'])
        if sum_ab==0: # 경기 기록은 존재하지만 타수(ab)가 0
          ps+=1
          continue
        sum_hit = sum(hrdata['hit'])
        recent_days_avg[ps] = sum_hit/sum_ab
        ps+=1

  result_Dataframe = pd.DataFrame()
  result_Dataframe['day'] = recent_days_day
  result_Dataframe['name'] = recent_days_hitter
  result_Dataframe['recent_days_avg'] = recent_days_avg
  result_Dataframe['double'] = recent_days_double

  return result_Dataframe

In [22]:
recent_days_data = make_recent_days(hitter, data)

100%|██████████| 90/90 [00:21<00:00,  4.15it/s]


# 결과 확인

삼성의 강민호로 확인 진행

In [25]:
hitter.loc[hitter['Team']=='삼성']

,Team,name
63,삼성,박해민
64,삼성,러프
65,삼성,김헌곤
66,삼성,이원석
67,삼성,구자욱
68,삼성,김상수
69,삼성,강민호
70,삼성,박한이
71,삼성,손주인


In [46]:
recent_days_data.loc[recent_days_data['name']=='강민호'].loc[8791:8795,:]

,day,name,recent_days_avg,double
8791,20180405,강민호,0.250000,0
8792,20180407,강민호,0.333333,0
8793,20180408,강민호,0.312500,0
8794,20180410,강민호,0.272727,0
8795,20180411,강민호,0.200000,0


In [52]:
data.loc[data['name']=='강민호',:].loc[8789:8795,:]

,name,day,vs,result,bat_order,position,start_member,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA,double
8789,강민호,20180403,@NC,L 3:4,6,C,True,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.250,NaN,0.357,0.680,20,1.26,-0.95,-0.081,0.0
8790,강민호,20180404,@NC,L 1:4,6,C,True,4,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0.281,NaN,0.375,0.718,21,0.89,0.69,0.093,0.0
8791,강민호,20180405,@NC,W 4:1,5,C,True,4,0,2,1,0,0,3,0,0,0,0,0,0,2,0,0,0,0.306,NaN,0.417,0.776,17,1.58,0.18,0.035,0.0
8792,강민호,20180407,@SK,L 3:4,5,C,True,4,0,1,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0.300,NaN,0.400,0.741,14,1.85,0.15,0.043,0.0
8793,강민호,20180408,@SK,W 12:4,5,C,True,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.279,NaN,0.372,0.705,14,0.43,0.87,0.035,0.0
8794,강민호,20180410,두산,L 1:8,5,C,True,3,0,1,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0.283,NaN,0.391,0.725,10,0.61,0.20,0.021,0.0
8795,강민호,20180411,두산,L 6:7,5,C,True,5,1,1,1,0,0,2,1,0,0,0,0,0,3,0,0,0,0.275,NaN,0.392,0.714,24,1.82,0.36,-0.072,0.0


In [56]:
# 자료 확인
# 2018년 4월 8일 기준 2018년 4월 3일부터 7일까지의 타율
print((0+2+2+1)/(4+4+4+4))

# 2018년 4월 10일 기준 2018년 4월 5일부터 9일까지의 타율
print((2+1+0)/(4+4+3))

# 2018년 4월 11일 기준 2018년 4월 6일부터 10일까지의 타율
print((1+0+1)/(4+3+3))

0.3125
0.2727272727272727
0.2
